In [1]:
import dask
import dask.threaded
import dask.multiprocessing
from dask.distributed import Client

c = Client()

c

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


In [6]:
import xarray as xr 
import dask 
import numpy as np 
import os 
import time 
import glob
from datetime import date
import pandas as pd

today=date.today()

In [7]:
## path à modifier

#data_dir = ''
gridfile='/scratch/cnt0024/hmg2840/albert7a/eORCA1/eORCA1-I/IPSLCM6ALR_eORCA1.2_mesh_mask.nc'
dsgrid=xr.open_dataset(gridfile,chunks={'x':1000,'y':1000})

In [8]:
## path à modifier
boxes_LS=pd.read_csv('/home/albert7a/git/formation_ANNA/make_boxes/boxes_LS_1x1_eNATL60.csv',sep = '\t',index_col=0)
lonmin_LS=boxes_LS['lonmin']
lonmax_LS=boxes_LS['lonmax']
latmin_LS=boxes_LS['latmin']
latmax_LS=boxes_LS['latmax']
box_name=boxes_LS.index

In [9]:
def dx_var(data,e1):
    dx_var = (data.shift(x=-1) - data)/e1
    return dx_var

def dy_var(data,e2):
    dy_var = (data.shift(y=-1) - data)/e2
    return dy_var

def dz_var(data,e3,dimdep):
    if dimdep == 'deptht':
        dz_var = (data.shift(deptht=-1) - data)/e3
    if dimdep == 'depthu':
        dz_var = (data.shift(depthu=-1) - data)/e3
    if dimdep == 'depthv':
        dz_var = (data.shift(depthv=-1) - data)/e3
    if dimdep == 'depthw':
        dz_var = (data.shift(depthw=-1) - data)/e3
    return dz_var
    

In [ ]:
def compute_buoy(t,s):
    rau0  = 1000
    grav  = 9.81
    buoy= -1*(grav/rau0)*sigma0(t,s)
    return buoy

def sigma0(t,s):
    zrau0=1000
    zsr=np.sqrt(np.abs(s))
    zs=s
    zt=t
    zr1 = ( ( ( ( 6.536332e-9*zt-1.120083e-6 )*zt+1.001685e-4)*zt - 9.095290e-3 )*zt+6.793952e-2 )*zt+999.842594
    zr2= ( ( ( 5.3875e-9*zt-8.2467e-7 )*zt+7.6438e-5 ) *zt - 4.0899e-3 ) *zt+0.824493
    zr3= ( -1.6546e-6*zt+1.0227e-4 ) *zt-5.72466e-3
    zr4= 4.8314e-4
    sigma0=( zr4*zs + zr3*zsr + zr2 ) *zs + zr1 - zrau0
    return sigma0


In [ ]:
## change name of the files if organization is different
filetyps = {'buoyancy' : 'gridT','votemper' : 'gridT', 'vosaline' : 'gridS','vozocrtx' : 'gridU', 'vomecrty' : 'gridV','vovecrtz' : 'gridW'}
filedeps = {'buoyancy' : 'deptht','votemper' : 'deptht','vosaline' : 'deptht','vozocrtx' : 'depthu', 'vomecrty' : 'depthv','vovecrtz':'depthw'}
filee1 = {'buoyancy' : 'e1t','votemper' : 'e1t','vosaline' : 'e1t','vozocrtx' : 'e1u', 'vomecrty' : 'e1v','vovecrtz':'e1f'}
filee2 = {'buoyancy' : 'e2t','votemper' : 'e2t','vosaline' : 'e2t','vozocrtx' : 'e2u', 'vomecrty' : 'e2v','vovecrtz':'e2f'}
filee3 = {'buoyancy' : 'e3t_0','votemper' : 'e3t_0','vosaline' : 'e3t_0','vozocrtx' : 'e3u_0', 'vomecrty' : 'e3v_0','vovecrtz':'e3w_0'}



In [ ]:
def compute_all_profiles(var,date,ibox):
    if var == 'buoyancy':

## change name of files
        filenameT = sorted(glob.glob(data_dir+'*/eNATL60-BLBT02_1h_*_gridT_'+date+'-'+date+'.nc'))
        fileT=filenameT[0]
        dsT=xr.open_dataset(fileT)
        data_boxT=dsT['votemper'][:,:,jmin_LS[ibox]-10:jmax_LS[ibox]+10,imin_LS[ibox]-10:imax_LS[ibox]+10]
## change name of files
        filenameS = sorted(glob.glob(data_dir+'*/eNATL60-BLBT02_1h_*_gridS_'+date+'-'+date+'.nc'))
        fileS=filenameS[0]
        dsS=xr.open_dataset(fileS)
        nav_lat=dsT['nav_lat']
        nav_lon=dsT['nav_lat']
        domain = ((lonmin[ibox]+lonmax[ibox])/2-0.5<navlon) * (navlon<(lonmin[ibox]+lonmax[ibox])/2+0.5) * ((latmin[ibox]+latmax[ibox])/2-0.5<navlat) * (navlat<(latmin[ibox]+latmax[ibox])/2+0.5)
        where = np.where(domain)
        vlats = navlat[where]
        vlons = navlon[where]
        #get indice
        jmin = where[0][vlats.argmin()]
        jmax = where[0][vlats.argmax()]
        imin = where[1][vlons.argmin()]
        imax = where[1][vlons.argmax()]

        data_boxS=dsS['vosaline'][:,:,jmin-10:jmax+10,imin-10:imax+10]
        data_box=compute_buoy(data_boxT,data_boxS)
        attrs=data_boxT.attrs
        attrs['standard_name']='Buoyancy'
        attrs['long_name']='Buoyancy'
        attrs['units']='m/s2'
    else:
## change name of files
        filename = sorted(glob.glob(data_dir+'*/eNATL60-BLBT02_1h_*_'+filetyps[var]+'_'+date+'-'+date+'.nc'))
        file=filename[0]
        ds=xr.open_dataset(file)
        data_box=ds[str(var)][:,:,jmin-10:jmax+10,imin-10:imax+10]
        attrs=data_box.attrs
        
    e1=dsgrid[str(filee1[var])][0,jmin-10:jmax+10,imin-10:imax+10]
    e2=dsgrid[str(filee2[var])][0,jmin-10:jmax+10,imin-10:imax+10]
    e3=dsgrid[str(filee3[var])][0,jmin-10:jmax+10,imin-10:imax+10]
    data_dx=dx_var(data_box,e1)
    data_dy=dy_var(data_box,e2)
    data_dz=dz_var(data_box,e3,filedeps[var])
    profile_data=data_box[:,:,10:-10,10:-10].mean(dim={'x','y','time_counter'})
    profile_data_dx=data_dx[:,:,10:-10,10:-10].mean(dim={'x','y','time_counter'})
    profile_data_dy=data_dy[:,:,10:-10,10:-10].mean(dim={'x','y','time_counter'})
    profile_data_dz=data_dz[:,:,10:-10,10:-10].mean(dim={'x','y','time_counter'})
    return profile_data,profile_data_dx,profile_data_dy,profile_data_dz,attrs


In [ ]:
def compute_all_profiles_all_var(date,ibox):
    list_dataset=[]
    for var in ['votemper','vosaline','vozocrtx','vomecrty','vovecrtz','buoyancy']:
        print('compute profile and dx,dy,dz of '+var)
        profile_data,profile_data_dx,profile_data_dy,profile_data_dz,attrs=compute_all_profiles(var,'20090701',0)
        dataset=profile_data.to_dataset(name=var)
        dataset[var].attrs=attrs
        dataset[var].attrs['standard_name']=attrs['standard_name']
        dataset[var].attrs['long_name']=attrs['long_name']
        dataset[var].attrs['units']=attrs['units']
        list_dataset.append(dataset)
        dataset=profile_data_dx.to_dataset(name='dx'+var)
        dataset['dx'+var].attrs=attrs
        dataset['dx'+var].attrs['standard_name']='dx gradient of '+attrs['standard_name']
        dataset['dx'+var].attrs['long_name']='dx_'+attrs['long_name']
        dataset['dx'+var].attrs['units']=attrs['units']
        list_dataset.append(dataset)
        dataset=profile_data_dy.to_dataset(name='dy'+var)
        dataset['dy'+var].attrs=attrs
        dataset['dy'+var].attrs['standard_name']='dy gradient of '+attrs['standard_name']
        dataset['dy'+var].attrs['long_name']='dy_'+attrs['long_name']
        dataset['dy'+var].attrs['units']=attrs['units']
        list_dataset.append(dataset)
        dataset=profile_data_dz.to_dataset(name='dz'+var)
        dataset['dz'+var].attrs=attrs
        dataset['dz'+var].attrs['standard_name']='dz gradient of '+attrs['standard_name']
        dataset['dz'+var].attrs['long_name']='dz_'+attrs['long_name']
        dataset['dz'+var].attrs['units']=attrs['units']
        list_dataset.append(dataset)
    print('merging all datasets')
## change path to output profile
    profile_name='/scratch/cnt0024/hmg2840/albert7a/eNATL60/eNATL60-BLBT02-S/ANNA/LS/eNATL60'+box_name[ibox]+'-BLBT02_y'+date[0:4]+'m'+date[4:6]+'d'+date[6:9]+'_predictors-profiles.nc'
    big_dataset=xr.merge(list_dataset) 
    big_dataset.attrs['global_attribute']= 'predictors profiles averaged over 24h and in '+box_name[ibox]+' computed on occigen with dask-jobqueue '+str(today)
    print('writing to netcdf')
    big_dataset.to_netcdf(path=profile_name,mode='w')


In [ ]:
for k in np.arange(len(box_name)):
    compute_all_profiles_all_var('20090701',k)
